COGS 160 Auto-Grader Notebook for Architect Assignments

Imports

In [1]:
import os
import re
import json
import fitz  
from PIL import Image
from io import BytesIO
from urllib.parse import urlparse
import spacy
import google.generativeai as genai
from dotenv import load_dotenv
from IPython.display import display
nlp = spacy.load("en_core_web_sm")

/opt/anaconda3/envs/langchain_setup_llama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configure Gemini

In [2]:
from dotenv import load_dotenv
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY) 
text_model = genai.GenerativeModel("gemini-1.5-pro-latest")
vision_model = genai.GenerativeModel("gemini-pro-vision")

In [ ]:
rubric = {
    "architect_selection": 5,
    "organization": 5,
    "biographical_content": 5,
    "bio_references": 5,
    "image_quality": 5,
    "image_citations": 5,
    "coverage_10_buildings": 5,
    "personal_bio_photo": 5,
    "presentation_polish": 5
}

def extract_text_from_pdf(pdf_path):
    print(f"🔍 Extracting text from: {pdf_path}")
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    print("✔ Extracted text from PDF")
    return text

def extract_images_from_pdf(pdf_path, min_width=1200, save_folder="/Users/tanishqsingh/Desktop/XR_Lab/Extracted_images"):
    print(f"🔍 Extracting images from: {pdf_path}")
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    image_data = []
    with fitz.open(pdf_path) as doc:
        for page_index in range(len(doc)):
            images = doc[page_index].get_images(full=True)
            for img_index, img in enumerate(images):
                xref = img[0]
                base_image = doc.extract_image(xref)
                img_bytes = base_image["image"]
                img_pil = Image.open(BytesIO(img_bytes)).convert("RGB")
                width, height = img_pil.size
                img_pil.save(os.path.join(save_folder, f"page{page_index+1}_img{img_index+1}.png"))
                image_data.append({
                    "page": page_index + 1,
                    "width": width,
                    "height": height,
                    "image": img_pil,
                    "is_high_res": width >= min_width
                })
    print(f"✔ Extracted {len(image_data)} images from PDF")
    return image_data

def extract_references_from_text(text):
    print("🔍 Extracting references from text")
    lines = text.split("\n")
    references = [line for line in lines if re.search(r"\(\d{4}\)", line) and any(x in line.lower() for x in ["doi", "https://", "archdaily", "e-architect"])]
    return references

def gemini_full_rubric_eval(text, architect_name, debug=True):
    print("🔍 Gemini evaluating extended rubric")
    prompt = f"""
You are grading a student submission for an architecture assignment on the architect {architect_name}.
This project includes a biography, building analysis, and images.
Evaluate the following 9 categories from 1–5 as per this rubric:

1. Architect Selection & Scope
2. Organization & Document Setup
3. Biographical Content (750 words)
4. Citation of Architect Biography
5. Selection & Quality of Images
6. Image Citation & Attribution
7. Coverage of 10 Famous Buildings
8. Personal Bio & Photo
9. Overall Completeness & Presentation

Provide a paragraph of feedback for each and return the score in parentheses at the end of each line (e.g., (3/5)).
"""
    response = text_model.generate_content([prompt, text])
    if debug:
        print(response.text)

    scores = {}
    for key, label in {
        "architect_selection": "Architect Selection",
        "organization": "Organization",
        "biographical_content": "Biographical Content",
        "bio_references": "Citation of Architect Biography",
        "image_quality": "Selection & Quality of Images",
        "image_citations": "Image Citation",
        "coverage_10_buildings": "Coverage of 10 Famous Buildings",
        "personal_bio_photo": "Personal Bio",
        "presentation_polish": "Overall Presentation"
    }.items():
        match = re.search(rf"\*\*{re.escape(label)}.*?\((\d)/5\)", response.text)
        if match:
            scores[key] = {"score": int(match.group(1))}
        else:
            scores[key] = {"score": 0}

    return scores

def evaluate_image_relevance(image_data, architect_name, debug=False):
    print("🔍 Evaluating image relevance using Gemini Vision")
    total_score = 0
    count = 0
    for img in image_data:
        prompt = f"Is this image visually related to a building designed by {architect_name}? Score from 1 (not relevant) to 5 (highly relevant), and explain."
        try:
            response = vision_model.generate_content([img["image"], prompt])
            if debug:
                print(f"📷 Page {img['page']} Image Relevance Feedback:\n{response.text}\n")
            match = re.search(r"(\d)/5", response.text)
            if match:
                total_score += int(match.group(1))
                count += 1
        except Exception as e:
            print(f"⚠️ Error processing image on page {img['page']}: {e}")
    avg_score = round(total_score / count, 2) if count else 0
    return {"avg_score": avg_score, "score": min(round(avg_score), 5)}

def run_autograder(pdf_path, architect_name):
    print("🚀 Starting pipeline")
    doc_text = extract_text_from_pdf(pdf_path)
    images = extract_images_from_pdf(pdf_path)
    references = extract_references_from_text(doc_text)

    rubric_scores = gemini_full_rubric_eval(doc_text, architect_name, debug=True)
    image_relevance = evaluate_image_relevance(images, architect_name, debug=True)

    rubric_scores["image_quality"]["vision_relevance"] = image_relevance

    final_score = sum(item["score"] for item in rubric_scores.values())
    grade = "A" if final_score >= 40 else "B" if final_score >= 35 else "C" if final_score >= 30 else "D"

    return {
        "scorecard": rubric_scores,
        "final_score": final_score,
        "grade": grade
    }

In [ ]:
result = run_autograder("/Users/tanishqsingh/Downloads/cogs160submisson1.pdf", "Bjarke Ingels")
print(json.dumps(result, indent=2))


🚀 Starting pipeline
🔍 Extracting text from: your_file_path.pdf


FileNotFoundError: no such file: 'your_file_path.pdf'